# Addis Ababa: Temporal Patterns Analysis (Daily 9am-Resampled Data)

This notebook covers daily, weekly, and seasonal patterns in BC concentrations using **daily 9am-to-9am resampled** data.

## Tasks Covered:
1. **Monthly patterns** - monthly BC variation by wavelength
2. **Weekly patterns** - weekday vs weekend analysis
3. **Ethiopian seasonal patterns** - BC by Dry/Belg/Kiremt seasons
4. **Extreme events analysis** - high BC day characterization
5. **Day-to-day rate of change** - BC buildup and decay patterns

---

## Setup and Imports

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import calendar

# Add scripts folder to path
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
scripts_path = os.path.join(notebook_dir, 'scripts')
if scripts_path not in sys.path:
    sys.path.insert(0, scripts_path)

from config import SITES
from data_matching import load_etad_factors_with_filter_ids
print("Loaded config and data_matching")

try:
    from plotting import PlotConfig
    print("Loaded plotting utilities")
except ImportError:
    print("Plotting utilities not found - will define inline")

# Configure matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

# Create output directories
def setup_directories():
    dirs = {
        'plots': 'output/plots/addis_ababa',
        'data': 'output/data/addis_ababa'
    }
    for dir_path in dirs.values():
        os.makedirs(dir_path, exist_ok=True)
    return dirs

dirs = setup_directories()
print("Setup complete!")

## Site Configuration

In [ ]:
# Addis Ababa site configuration (daily data)
ADDIS_CONFIG = {
    'name': 'Addis_Ababa',
    'timezone': 'Africa/Addis_Ababa',
    'wavelengths': {'UV': 375, 'Blue': 470, 'Green': 528, 'Red': 625, 'IR': 880},
    'bc_columns': [
        'UV BCc', 'Blue BCc', 'Green BCc', 'Red BCc', 'IR BCc'
    ],
    'primary_bc_col': 'IR BCc',
    'seasons': {
        'Dry Season': [10, 11, 12, 1, 2],
        'Belg Rainy Season': [3, 4, 5],
        'Kiremt Rainy Season': [6, 7, 8, 9]
    }
}

SEASONS_ORDER = ['Dry Season', 'Belg Rainy Season', 'Kiremt Rainy Season']
SEASON_COLORS = {'Dry Season': '#E67E22', 'Belg Rainy Season': '#27AE60', 'Kiremt Rainy Season': '#3498DB'}

print(f"Site: {ADDIS_CONFIG['name']}")
print(f"Primary BC column: {ADDIS_CONFIG['primary_bc_col']}")

## Data Loading

Uses the same loading function from the source apportionment notebook.

In [ ]:
DATA_FILEPATH = "/Users/ahmadjalil/github/aethmodular/FTIR_HIPS_Chem/processed_sites/df_Addis_Ababa_9am_resampled.pkl"

def load_aethalometer_addis(filepath):
    """
    Load and preprocess Addis Ababa daily (9am-resampled) aethalometer data from pickle file.
    """
    df = pd.read_pickle(filepath)
    
    # Set datetime index
    df['datetime_local'] = pd.to_datetime(df['datetime_local'])
    df.set_index('datetime_local', inplace=True)
    df.sort_index(inplace=True)
    
    # Convert from ng/m³ to µg/m³
    bc_columns = ADDIS_CONFIG['bc_columns']
    for col in bc_columns:
        if col in df.columns:
            df[col] = df[col] / 1000
    
    # Add time-based columns
    df['Month'] = df.index.month
    df['DayOfWeek'] = df.index.dayofweek
    df['DayOfYear'] = df.index.dayofyear
    
    # Add Ethiopian seasons
    df['Ethiopian_Season'] = df['Month'].map(lambda m: 
        'Dry Season' if m in ADDIS_CONFIG['seasons']['Dry Season'] else
        'Belg Rainy Season' if m in ADDIS_CONFIG['seasons']['Belg Rainy Season'] else
        'Kiremt Rainy Season'
    )
    
    # Handle outliers
    for col in bc_columns:
        if col in df.columns:
            df.loc[df[col] < 0, col] = np.nan
            mean = df[col].mean()
            std = df[col].std()
            df.loc[df[col] > mean + 3*std, col] = np.nan
    
    return df

df = load_aethalometer_addis(DATA_FILEPATH)
print(f"Loaded {len(df):,} daily records")
print(f"Date range: {df.index.min()} to {df.index.max()}")

In [ ]:
# --- Load PMF Factor Contributions ---
FACTOR_TO_FRAC = {
    'GF3 (Charcoal)':              'charcoal_frac',
    'GF2 (Wood Burning)':          'wood_frac',
    'GF5 (Fossil Fuel Combustion)':'fossil_fuel_frac',
    'GF4 (Polluted Marine)':       'polluted_marine_frac',
    'GF1 (Sea Salt Mixed)':        'sea_salt_frac',
}

factors_df = load_etad_factors_with_filter_ids()
factors_df = factors_df.rename(columns=FACTOR_TO_FRAC)
frac_cols = list(FACTOR_TO_FRAC.values())

# Normalize to relative source contributions (raw GFs are PM2.5 mass fractions, not relative)
frac_sum = factors_df[frac_cols].sum(axis=1)
for col in frac_cols:
    factors_df[col] = factors_df[col] / frac_sum

factor_map = factors_df.set_index('date')[frac_cols]

merge_dates = df.index.normalize()
if merge_dates.tz is not None:
    merge_dates = merge_dates.tz_localize(None)

for col in frac_cols:
    df[col] = merge_dates.map(factor_map[col])

# Add dominant source
df['dominant_source'] = df[frac_cols].idxmax(axis=1).str.replace('_frac', '')
df['dominant_fraction'] = df[frac_cols].max(axis=1)

n_with = df[frac_cols].notna().any(axis=1).sum()
print(f"Factor data merged: {n_with} rows with factor data out of {len(df)} total")
print(f"Dominant fraction: mean={df['dominant_fraction'].dropna().mean():.1%}, "
      f"≥50%: {(df['dominant_fraction'] >= 0.50).sum()}, "
      f"≥30%: {(df['dominant_fraction'] >= 0.30).sum()}")

---

# Task 1: Monthly Patterns

**Goal**: Analyze monthly BC concentration patterns across wavelengths.

In [ ]:
def plot_monthly_pattern(df, bc_cols=['IR BCc', 'UV BCc'], title_suffix=""):
    """
    Plot average monthly pattern for specified BC columns.
    """
    fig, ax = plt.subplots(figsize=(12, 6))
    
    colors = ['darkred', 'purple', 'blue', 'green', 'orange']
    
    for i, col in enumerate(bc_cols):
        if col not in df.columns:
            continue
        monthly_avg = df.groupby('Month')[col].mean()
        monthly_std = df.groupby('Month')[col].std()
        
        wave = col.split()[0]
        wavelength = ADDIS_CONFIG['wavelengths'].get(wave, '')
        label = f'{wave} ({wavelength}nm)' if wavelength else col
        
        ax.plot(monthly_avg.index, monthly_avg.values, marker='o', 
                color=colors[i % len(colors)], linewidth=2, label=label)
        ax.fill_between(monthly_avg.index, 
                       monthly_avg - monthly_std, 
                       monthly_avg + monthly_std,
                       alpha=0.2, color=colors[i % len(colors)])
    
    ax.set_xlabel('Month', fontsize=12)
    ax.set_ylabel('BC Concentration (µg/m³)', fontsize=12)
    ax.set_title(f'Average Monthly Pattern of Black Carbon (Daily Data){title_suffix}', fontsize=14, fontweight='bold')
    ax.set_xticks(range(1, 13))
    ax.set_xticklabels([calendar.month_abbr[m] for m in range(1, 13)], rotation=45)
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig


def plot_monthly_by_season(df, bc_col='IR BCc'):
    """
    Plot monthly patterns separated by Ethiopian season.
    """
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    for idx, season in enumerate(SEASONS_ORDER):
        ax = axes[idx]
        season_data = df[df['Ethiopian_Season'] == season]
        
        monthly_avg = season_data.groupby('Month')[bc_col].mean()
        monthly_std = season_data.groupby('Month')[bc_col].std()
        
        ax.bar(monthly_avg.index, monthly_avg.values,
               color=SEASON_COLORS[season], alpha=0.7, edgecolor='black')
        ax.errorbar(monthly_avg.index, monthly_avg.values, yerr=monthly_std.values,
                   fmt='none', color='black', capsize=3)
        
        ax.set_xlabel('Month', fontsize=11)
        ax.set_ylabel('BC Concentration (µg/m³)' if idx == 0 else '', fontsize=11)
        ax.set_title(f'{season}', fontsize=12, fontweight='bold')
        ax.set_xticks(monthly_avg.index)
        ax.set_xticklabels([calendar.month_abbr[m] for m in monthly_avg.index], rotation=45)
        ax.grid(True, alpha=0.3, axis='y')
        
        n = len(season_data[bc_col].dropna())
        ax.text(0.95, 0.95, f'n={n:,} days',
                transform=ax.transAxes, fontsize=9, va='top', ha='right',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))
    
    plt.suptitle('Monthly BC Patterns by Ethiopian Season (Daily Data)', fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    return fig

print("="*80)
print("TASK 1: MONTHLY PATTERNS")
print("="*80)
fig1 = plot_monthly_pattern(df, bc_cols=['IR BCc', 'UV BCc'])
plt.savefig(os.path.join(dirs['plots'], 'monthly_pattern_daily.png'), dpi=150, bbox_inches='tight')
plt.show()

fig2 = plot_monthly_by_season(df)
plt.savefig(os.path.join(dirs['plots'], 'monthly_by_season_daily.png'), dpi=150, bbox_inches='tight')
plt.show()

---

# Task 2: Weekly Patterns

**Goal**: Analyze weekday vs weekend BC differences.

In [ ]:
def analyze_weekend_effect(df, bc_col='IR BCc'):
    """
    Calculate weekend effect (reduction) by season using daily data.
    """
    results = {}
    
    print("\nWeekend Effect Analysis (Daily Data):")
    print("=" * 60)
    
    for season in SEASONS_ORDER:
        season_data = df[df['Ethiopian_Season'] == season]
        
        weekday_data = season_data[season_data['DayOfWeek'] < 5][bc_col]
        weekend_data = season_data[season_data['DayOfWeek'] >= 5][bc_col]
        
        weekday_avg = weekday_data.mean()
        weekend_avg = weekend_data.mean()
        reduction = ((weekday_avg - weekend_avg) / weekday_avg) * 100
        
        # T-test for significance
        t_stat, p_value = stats.ttest_ind(weekday_data.dropna(), weekend_data.dropna())
        
        results[season] = {
            'weekday_avg': weekday_avg,
            'weekend_avg': weekend_avg,
            'reduction_pct': reduction,
            'p_value': p_value,
            'n_weekday': len(weekday_data.dropna()),
            'n_weekend': len(weekend_data.dropna())
        }
        
        sig = '*' if p_value < 0.05 else ''
        print(f"\n{season}:")
        print(f"  Weekday avg: {weekday_avg:.3f} µg/m³ (n={results[season]['n_weekday']:,} days)")
        print(f"  Weekend avg: {weekend_avg:.3f} µg/m³ (n={results[season]['n_weekend']:,} days)")
        print(f"  Reduction: {reduction:.1f}%{sig} (p={p_value:.3e})")
    
    return results


def plot_weekly_boxplot(df, bc_col='IR BCc'):
    """
    Create day-of-week boxplot by season.
    """
    day_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    fig, axes = plt.subplots(1, 3, figsize=(20, 6))
    
    for idx, season in enumerate(SEASONS_ORDER):
        ax = axes[idx]
        season_data = df[df['Ethiopian_Season'] == season]
        
        # Create boxplot data by day of week
        plot_data = []
        for dow in range(7):
            day_data = season_data[season_data['DayOfWeek'] == dow][bc_col].dropna()
            plot_data.append(day_data)
        
        bp = ax.boxplot(plot_data, labels=day_names, patch_artist=True, showfliers=False)
        
        # Color weekdays vs weekends
        for i, patch in enumerate(bp['boxes']):
            if i < 5:
                patch.set_facecolor('#AED6F1')
            else:
                patch.set_facecolor('#F5B7B1')
            patch.set_alpha(0.8)
        
        ax.set_title(f'{season}', fontsize=12, fontweight='bold')
        ax.set_xlabel('Day of Week')
        ax.set_ylabel('BC Concentration (µg/m³)' if idx == 0 else '')
        ax.grid(True, alpha=0.3, axis='y')
        
        # Add count
        n = len(season_data[bc_col].dropna())
        ax.text(0.95, 0.95, f'n={n:,} days',
                transform=ax.transAxes, fontsize=9, va='top', ha='right',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))
    
    plt.suptitle('Daily BC by Day of Week and Season', fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    return fig

print("="*80)
print("TASK 2: WEEKLY PATTERNS")
print("="*80)
weekend_results = analyze_weekend_effect(df)
fig = plot_weekly_boxplot(df)
plt.savefig(os.path.join(dirs['plots'], 'weekly_boxplot_daily.png'), dpi=150, bbox_inches='tight')
plt.show()

---

# Task 3: Ethiopian Seasonal Patterns

**Goal**: Comprehensive analysis of BC patterns across Ethiopian seasons.

In [ ]:
def analyze_seasonal_statistics(df, bc_col='IR BCc'):
    """
    Calculate detailed statistics for each Ethiopian season.
    """
    results = {}
    
    print("\nSeasonal Statistics (Daily Data):")
    print("=" * 80)
    
    for season in SEASONS_ORDER:
        season_data = df[df['Ethiopian_Season'] == season][bc_col].dropna()
        
        results[season] = {
            'n': len(season_data),
            'mean': season_data.mean(),
            'median': season_data.median(),
            'std': season_data.std(),
            'min': season_data.min(),
            'max': season_data.max(),
            'q25': season_data.quantile(0.25),
            'q75': season_data.quantile(0.75),
        }
        
        r = results[season]
        print(f"\n{season}:")
        print(f"  n = {r['n']:,} days")
        print(f"  Mean +/- Std: {r['mean']:.3f} +/- {r['std']:.3f} µg/m³")
        print(f"  Median [IQR]: {r['median']:.3f} [{r['q25']:.3f} - {r['q75']:.3f}] µg/m³")
        print(f"  Range: {r['min']:.3f} - {r['max']:.3f} µg/m³")
    
    return results


def plot_seasonal_boxplot(df, bc_col='IR BCc'):
    """
    Create boxplot comparing BC across Ethiopian seasons.
    """
    fig, ax = plt.subplots(figsize=(10, 6))
    
    plot_data = []
    for season in SEASONS_ORDER:
        season_data = df[df['Ethiopian_Season'] == season][bc_col].dropna()
        plot_data.append(season_data)
    
    bp = ax.boxplot(plot_data, labels=SEASONS_ORDER, patch_artist=True, showfliers=False)
    
    for patch, season in zip(bp['boxes'], SEASONS_ORDER):
        patch.set_facecolor(SEASON_COLORS[season])
        patch.set_alpha(0.7)
    
    ax.set_xlabel('Ethiopian Season', fontsize=12)
    ax.set_ylabel('BC Concentration (µg/m³)', fontsize=12)
    ax.set_title('Daily BC Distribution by Ethiopian Season', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')
    
    for i, season in enumerate(SEASONS_ORDER):
        n = len(df[df['Ethiopian_Season'] == season][bc_col].dropna())
        ax.text(i + 1, ax.get_ylim()[1] * 0.95, f'n={n:,}', ha='center', fontsize=9)
    
    plt.tight_layout()
    return fig


def plot_seasonal_timeseries(df, bc_col='IR BCc'):
    """
    Plot daily time series with seasonal shading and rolling average.
    """
    fig, ax = plt.subplots(figsize=(15, 6))
    
    ax.plot(df.index, df[bc_col], 'o', alpha=0.3, color='gray', markersize=3)
    
    # Rolling average (30-day)
    rolling_avg = df[bc_col].rolling(window=30, center=True, min_periods=7).mean()
    ax.plot(rolling_avg.index, rolling_avg, color='darkblue', linewidth=2, label='30-day rolling mean')
    
    # Add seasonal shading
    for season, months in ADDIS_CONFIG['seasons'].items():
        for date in df.index:
            if date.month in months:
                ax.axvspan(date, date + pd.Timedelta(days=1), alpha=0.1, 
                          color=SEASON_COLORS.get(season, 'gray'), linewidth=0)
    
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('BC Concentration (µg/m³)', fontsize=12)
    ax.set_title('Daily BC Time Series with Seasonal Context', fontsize=14, fontweight='bold')
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

print("="*80)
print("TASK 3: ETHIOPIAN SEASONAL PATTERNS")
print("="*80)
seasonal_stats = analyze_seasonal_statistics(df)
fig1 = plot_seasonal_boxplot(df)
plt.savefig(os.path.join(dirs['plots'], 'seasonal_boxplot_daily.png'), dpi=150, bbox_inches='tight')
plt.show()

fig2 = plot_seasonal_timeseries(df)
plt.savefig(os.path.join(dirs['plots'], 'seasonal_timeseries_daily.png'), dpi=150, bbox_inches='tight')
plt.show()

---

# Task 4: Extreme Events Analysis

**Goal**: Characterize high BC concentration days.

In [ ]:
def analyze_extreme_events(df, bc_col='IR BCc', percentile=95):
    """
    Identify and characterize extreme BC days.
    """
    threshold = df[bc_col].quantile(percentile / 100)
    extreme_mask = df[bc_col] > threshold
    extreme_events = df[extreme_mask].copy()
    
    print(f"\nExtreme Events Analysis (>{percentile}th percentile, Daily Data):")
    print("=" * 60)
    print(f"Threshold: {threshold:.3f} µg/m³")
    print(f"Total extreme days: {extreme_mask.sum():,}")
    
    # Seasonal distribution
    print("\nExtreme days by season:")
    seasonal_counts = extreme_events['Ethiopian_Season'].value_counts()
    for season in SEASONS_ORDER:
        count = seasonal_counts.get(season, 0)
        pct = count / extreme_mask.sum() * 100
        print(f"  {season}: {count:,} ({pct:.1f}%)")
    
    # Day of week distribution
    print("\nExtreme days by day of week:")
    day_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    dow_counts = extreme_events['DayOfWeek'].value_counts().sort_index()
    for dow, count in dow_counts.items():
        print(f"  {day_names[dow]}: {count:,} days")
    
    return extreme_events, threshold


def plot_extreme_event_distribution(df, extreme_events, threshold, bc_col='IR BCc'):
    """
    Visualize extreme event distribution by day of week and season.
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Day of week distribution
    ax = axes[0]
    day_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    dow_counts = extreme_events['DayOfWeek'].value_counts().sort_index()
    colors = ['#AED6F1' if i < 5 else '#F5B7B1' for i in range(7)]
    ax.bar(range(7), [dow_counts.get(i, 0) for i in range(7)], 
           color=colors, alpha=0.7, edgecolor='black')
    ax.set_xticks(range(7))
    ax.set_xticklabels(day_names)
    ax.set_xlabel('Day of Week', fontsize=11)
    ax.set_ylabel('Number of Extreme Days', fontsize=11)
    ax.set_title('Extreme Days by Day of Week', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='y')
    
    # Seasonal distribution
    ax = axes[1]
    seasonal_counts = [extreme_events[extreme_events['Ethiopian_Season'] == s].shape[0] for s in SEASONS_ORDER]
    colors = [SEASON_COLORS[s] for s in SEASONS_ORDER]
    ax.bar(SEASONS_ORDER, seasonal_counts, color=colors, alpha=0.7, edgecolor='black')
    ax.set_xlabel('Ethiopian Season', fontsize=11)
    ax.set_ylabel('Number of Extreme Days', fontsize=11)
    ax.set_title('Extreme Days by Season', fontsize=12, fontweight='bold')
    ax.tick_params(axis='x', rotation=15)
    ax.grid(True, alpha=0.3, axis='y')
    
    plt.suptitle(f'Extreme BC Days (>{threshold:.2f} µg/m³)', fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    return fig

print("="*80)
print("TASK 4: EXTREME EVENTS ANALYSIS")
print("="*80)
extreme_events, threshold = analyze_extreme_events(df)
fig = plot_extreme_event_distribution(df, extreme_events, threshold)
plt.savefig(os.path.join(dirs['plots'], 'extreme_events_daily.png'), dpi=150, bbox_inches='tight')
plt.show()

---

# Task 5: Day-to-Day Rate of Change Analysis

**Goal**: Analyze daily BC concentration buildup and decay patterns.

In [ ]:
def analyze_rate_of_change(df, bc_col='IR BCc'):
    """
    Calculate and analyze day-to-day rate of change in BC concentrations.
    """
    df_temp = df.copy()
    df_temp['BC_RateOfChange'] = df_temp[bc_col].diff()
    
    print("\nDay-to-Day Rate of Change Analysis:")
    print("=" * 60)
    
    roc = df_temp['BC_RateOfChange'].dropna()
    print(f"Overall statistics:")
    print(f"  Mean rate: {roc.mean():.4f} µg/m³/day")
    print(f"  Std: {roc.std():.4f} µg/m³/day")
    
    # Monthly patterns
    print("\nAverage rate by month (positive = increasing):")
    monthly_roc = df_temp.groupby('Month')['BC_RateOfChange'].mean()
    for month in range(1, 13):
        if month in monthly_roc.index:
            direction = "+" if monthly_roc[month] > 0 else ""
            print(f"  {calendar.month_abbr[month]}: {direction}{monthly_roc[month]:.4f} µg/m³/day")
    
    # Seasonal patterns
    print("\nAverage rate by season:")
    seasonal_roc = df_temp.groupby('Ethiopian_Season')['BC_RateOfChange'].mean()
    for season in SEASONS_ORDER:
        if season in seasonal_roc.index:
            direction = "+" if seasonal_roc[season] > 0 else ""
            print(f"  {season}: {direction}{seasonal_roc[season]:.4f} µg/m³/day")
    
    return df_temp['BC_RateOfChange'], monthly_roc


def plot_rate_of_change(df, roc, monthly_roc, bc_col='IR BCc'):
    """
    Visualize day-to-day rate of change patterns.
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Monthly average rate of change
    ax = axes[0]
    months = sorted(monthly_roc.index)
    values = [monthly_roc[m] for m in months]
    colors = ['#2ECC71' if v > 0 else '#E74C3C' for v in values]
    ax.bar(months, values, color=colors, alpha=0.7, edgecolor='black')
    ax.axhline(y=0, color='black', linestyle='-', linewidth=1)
    ax.set_xlabel('Month', fontsize=11)
    ax.set_ylabel('Avg Rate of Change (µg/m³/day)', fontsize=11)
    ax.set_title('Average Monthly Rate of Change', fontsize=12, fontweight='bold')
    ax.set_xticks(months)
    ax.set_xticklabels([calendar.month_abbr[m] for m in months], rotation=45)
    ax.grid(True, alpha=0.3, axis='y')
    
    # Distribution of rate of change
    ax = axes[1]
    roc_clean = roc.dropna()
    clip_val = roc_clean.std() * 3
    ax.hist(roc_clean.clip(-clip_val, clip_val), bins=50, alpha=0.7, color='steelblue', 
            edgecolor='black', linewidth=0.5)
    ax.axvline(x=0, color='red', linestyle='--', linewidth=2)
    ax.set_xlabel('Rate of Change (µg/m³/day)', fontsize=11)
    ax.set_ylabel('Frequency', fontsize=11)
    ax.set_title('Distribution of Day-to-Day Rate of Change', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3)
    
    plt.suptitle('BC Concentration Day-to-Day Rate of Change Analysis', fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    return fig

print("="*80)
print("TASK 5: DAY-TO-DAY RATE OF CHANGE ANALYSIS")
print("="*80)
roc, monthly_roc = analyze_rate_of_change(df)
fig = plot_rate_of_change(df, roc, monthly_roc)
plt.savefig(os.path.join(dirs['plots'], 'rate_of_change_daily.png'), dpi=150, bbox_inches='tight')
plt.show()

---

# Summary

## Functions Defined:
- `plot_monthly_pattern()` - Monthly BC patterns
- `plot_monthly_by_season()` - Monthly patterns by Ethiopian season
- `analyze_weekend_effect()` - Weekday vs weekend analysis
- `plot_weekly_boxplot()` - Day-of-week boxplots by season
- `analyze_seasonal_statistics()` - Seasonal statistical summary
- `plot_seasonal_boxplot()` - Seasonal distribution comparison
- `analyze_extreme_events()` - High BC day characterization
- `analyze_rate_of_change()` - Day-to-day BC change patterns

## Data:
- Daily (9am-to-9am resampled) aethalometer data

In [ ]:
print("="*80)
print("NOTEBOOK 02 (DAILY): TEMPORAL PATTERNS COMPLETE")
print("="*80)